#Inference of pre-trained models using HuggingFace library (on GPU)
The trained tensorflow models have been ported to huggingface but they don't match the reported results (are actually lower) due to some implementation differences between huggingface and tensorflow.

Use the AACT_MOIE_Tensorflow sheet if you want to use the best models or want to train the models from scratch - [Colab link](https://colab.research.google.com/drive/1gzVL5ZGHRCKJJcwj8DSghbmYjUIsC2VM?usp=sharing).

##Install Requirements

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install ipdb
!pip install --upgrade gdown
!pip install sklearn
!git clone https://github.com/dair-iitd/moie.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 120 kB 69.6 MB/s 
     |████████████████████████████████| 6.6 MB 38.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 793 kB 5.4 MB/s 
     |████████████████████████████████| 1.6 MB 52.6 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=e3cbe0763d9acc1ae7c6714ec655fcbc0e713cc12a40ff82475a66bc7e88e0c0
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: ipython
    Found existing installation: ipython 7.9.0
 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title Fill in parameters
language = 'en' #@param ["en", "es", "pt", "hi", "te", "zh"] {allow-input: true}
import os
os.environ['LANG'] = language
device = 'cuda:0' #@param {type:"string"}
evaluate_carb = False #@param {type:"boolean"}
if not evaluate_carb:
  input_file = 'file_path_containing_sentences' #@param {type:"string"}
else:
  input_file = f'moie/carb/data/{language}_test.input'

##Download models

In [ ]:
%%shell

if [ "$LANG" = "en" ]; then
  gdown 1MmHBqWApqNhRwuNPw3mTpZnY0wdghqZy ## EN
  unzip aact_moie_gen2oie_en.zip
elif [ "$LANG" = "es" ]; then
  gdown 1m3zzAsktqx_6V4aj4LUq3cpQEH_MH-G1 ## ES
  unzip aact_moie_gen2oie_es.zip
elif [ "$LANG" = "pt" ]; then
  gdown 1__AqeNNAsdIzL_Zpjko1BF3nexwAUCQV ## PT
  unzip aact_moie_gen2oie_pt.zip
elif [ "$LANG" = "hi" ]; then
  gdown 130EwTdJa5O278tD4sQsRN0rTyijWoI6z ## HI
  unzip aact_moie_gen2oie_hi.zip
elif [ "$LANG" = "te" ]; then
  gdown 1geLobUxGwOTtY54ogblDIyUa2HlAtv-M ## TE
  unzip aact_moie_gen2oie_te.zip
elif [ "$LANG" = "zh" ]; then
  gdown 13VCbjE--4-EiX8adExgPHcl1Tsy_MX-T ## ZH
  unzip aact_moie_gen2oie_zh.zip
fi

##Load the models

In [ ]:
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

s1_path = f'aact_moie_gen2oie_{language}/stage1/'
s1_model = AutoModelForSeq2SeqLM.from_pretrained(s1_path).to(device)

s2_path = f'aact_moie_gen2oie_{language}/stage2/'
s2_model = AutoModelForSeq2SeqLM.from_pretrained(s2_path).to(device)

tokenizer = AutoTokenizer.from_pretrained('google/mt5-base')

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

##Load Input Sentences on which OpenIE has to be run

In [ ]:
sentences = open(input_file).readlines()
sentences = [s.strip() for s in sentences]

##Run the code

In [ ]:
import math
from tqdm import tqdm

s1_results = []
batch_size = 32
num_batches = math.ceil(len(sentences)/batch_size)
print('Stage-1 prediction:')
for k in tqdm(range(num_batches)):
  sentences_k = sentences[k*batch_size:(k+1)*batch_size]

  s1_inputs_toks = tokenizer(sentences_k, return_tensors="pt", padding=True)
  s1_inputs_toks = s1_inputs_toks['input_ids'].to(device)
  preds_k = s1_model.generate(s1_inputs_toks, max_length=50, num_beams=1, early_stopping=True)
  s1_results_k = tokenizer.batch_decode(preds_k, skip_special_tokens=True)
  s1_results.extend(s1_results_k)

all_outputs = []
s2_inputs, s2_input_sentences = [], []
num_relations = []
nrs = 0
num_relations.append(0)
for sentence_i, sentence in enumerate(sentences):
    for rel in s1_results[sentence_i].split('<r>'):
        s2_input = rel.strip() + ' <r> ' + sentence
        s2_inputs.append(s2_input)
        nrs += 1
    num_relations.append(nrs)

s2_results = []
batch_size = 8
num_batches = math.ceil(len(s2_inputs)/batch_size)
for k in tqdm(range(num_batches)):
    s2_input_k = s2_inputs[k*batch_size:(k+1)*batch_size]
    s2_inputs_toks = tokenizer(s2_input_k, return_tensors="pt", padding=True)
    s2_inputs_toks = s2_inputs_toks['input_ids'].to(device)

    preds_k = s2_model.generate(s2_inputs_toks, max_length=128, num_beams=1, early_stopping=True)
    s2_results_k = tokenizer.batch_decode(preds_k, skip_special_tokens=True)
    s2_results.extend(s2_results_k)

s2r = []
for ri, result in enumerate(s2_results):
    if ri in num_relations:
        s2r.append([result])
    else:
        s2r[-1].append(result)

for sentence, s2_result in zip(sentences, s2r):
    def flatten(t):
        return [item for sublist in t for item in sublist if item.strip()]
    extractions = []
    for e1 in s2_result:
        for e2 in e1.split('<e>'):
            if e2.strip():
                extractions.append(e2)
    extractions = set(extractions)
    all_outputs.append([sentence, extractions])


Stage-1 prediction:


100%|██████████| 293/293 [04:31<00:00,  1.08it/s]


## Print the generated extractions

In [ ]:
import re

f = open('extractions','w')

for sentence, extractions in all_outputs:
  if not evaluate_carb:
    print('Sentence: ')
    f.write(sentence+'\n')
    print(sentence)
    print('Extractions: ')
    print('\n'.join(extractions))
    f.write('\n'.join(extractions)+'\n\n')
    print('\n')
  else:
    for extraction in extractions:
      time = re.findall('<t>(.*?)</t>', extraction)
      time = ' '.join([t.strip() for t in time]).strip()
      loc = re.findall('<l>(.*?)</l>', extraction)
      loc = ' '.join([t.strip() for t in loc]).strip()
      arg2 = re.findall('<a2>(.*?)</a2>', extraction)
      arg2 = ' '.join([t.strip() for t in arg2]).strip()
      if loc:
        arg2 = arg2+' '+loc
      if time:
        arg2 = arg2+' '+time
      arg2 = arg2.strip()

      arg1 = re.findall('<a1>(.*?)</a1>', extraction)
      arg1 = ' '.join([t.strip() for t in arg1]).strip()
      rel = re.findall('<r>(.*?)</r>', extraction)
      rel = ' '.join([t.strip() for t in rel]).strip()
      extraction = f'<arg1> {arg1} </arg1> <rel> {rel} </rel>'
      if arg2:
        extraction += f' <arg2> {arg2} </arg2>'
      f.write(f'{sentence}\t{extraction}\t1\n')
f.close()

In [ ]:
## Run only in case of evaluating CaRB
!python moie/carb/carb.py --allennlp extractions --gold moie/carb/data/gold/${LANG}_test.tsv --out /dev/null

INFO:root:Writing PR curve of Allennlp to /dev/null
Num Final Sentences =  634
AUC: 0.3731	 Optimal (precision, recall, F1): (0.5407, 0.4843, 0.511)	Zero Conf (precision, recall, F1): (0.5407, 0.4843, 0.511)
